### In this notebook, I first generate faces using captions from COCO dataset using ERNIE-ViLG, and then, I using face detector to extract faces from the images generated. 

In [1]:
#!pip install translators --upgrade

Load caption data

In [2]:
import paddlehub as hub
import translators as ts
lines = open('face_captions.txt').read().splitlines()

/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/ernie_vil/feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,
/home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/paddlenlp/transformers/clip/feature_extraction.py:64: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10

In [3]:
import random
random.seed(42)
#Generate 1000 random numbers between 0 and 9999
# I plan to generate about 600 images, but may need to come back generate more if some of the images do not contain a good face
randomlist = random.sample(range(0, 9999), 1100)

In [4]:
len(randomlist)

1100

In [5]:
randomlist[:10]

[1824, 409, 4506, 4012, 3657, 2286, 1679, 8935, 1424, 9674]

Start generating face images

In [9]:
import paddlehub as hub
import translators as ts
module = hub.Module(name="ernie_vilg")
for i in range(279, 400):
    prompt = ts.baidu(lines[randomlist[i]] + 'showing front face, like a photo in real life', professional_field='common')
    try: # for certain prompts, error message "暂不支持创作该内容，请修改后再试 (does not support this prompt, please modify)" was returned
        module.generate_image(text_prompts=prompt, style='写实风格',  output_dir='./generated_face_images_with_ERNIE-ViLG_version_2/',topk=1)
        print(i)      
    except:
        continue

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
279


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
280


  0%|          | 0/100 [00:00<?, ?%/s]

...

  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
352


  0%|          | 0/100 [00:00<?, ?%/s]

Saving Images...
Done
353
日访问量超限
日访问量超限


KeyboardInterrupt: 

In [ ]:
i

extract faces from the generated face images 

In [10]:
#set up folders for extracted faces 
# initialize COCO API for instance annotations
import os
dataDir = './'

image_folder = "extracted_generated_faces_version2"

savedirimage = dataDir + image_folder



if not os.path.exists(savedirimage):
	os.makedirs(savedirimage)

In [11]:
# install all packages
!pip install keras
!pip install tensorflow
from keras.models import load_model
!pip install mtcnn
# confirm mtcnn was installed correctly
import mtcnn
# print version
print(mtcnn.__version__)

import numpy as np
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import matplotlib.pyplot as plt


# Simple python package to shut up Tensorflow warnings and logs.
!pip install silence_tensorflow
import silence_tensorflow.auto

2022-11-28 18:25:00.684247: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 18:25:04.184810: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/xiaona/anaconda3/envs/paddle_env/lib/python3.8/site-packages/cv2/../../lib64:
2022-11-28 18:25:04.184894: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-11-28 18:25:18.750921: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnv

0.1.0


I did not use threshold as filtering criteria because this will gaurantee low FID score later. I still kept the dimension requirement. 

In [12]:
# extract a single face from a given photograph
def extract_face(filename, index, required_size=(250, 250)):
    # load image from file
    image = Image.open(filename)
    # convert to RGB, if needed
    image = image.convert('RGB')
    # convert to array
    pixels = np.asarray(image)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    if len(results) >=1:
        x1, y1, width, height = results[0]['box']
        if height-width>=15:
            # bug fix
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height
            # extract the face
            face = pixels[y1:y2, x1:x2]
            # resize pixels to the model size
            image = Image.fromarray(face)
            image = image.resize(required_size)
            face_array = np.asarray(image)
            plt.imsave(os.path.join(savedirimage, 'face_img_{}.jpg'.format(index)),face_array)
            return True
    return False

In [14]:
files = os.listdir('generated_face_images_with_ERNIE-ViLG_version_2/')

In [ ]:
index =0
for file in files:
    try:
        if extract_face(os.path.join('./generated_face_images_with_ERNIE-ViLG_version_2/',file), index = index):
            print(index)
            index+=1
    except:
        None

1/1 [==============================] - 0s 143ms/step
0
1/1 [==============================] - 0s 127ms/step
1
1/1 [==============================] - 0s 126ms/step
2
1/1 [==============================] - 0s 125ms/step
3
1/1 [==============================] - 0s 128ms/step
4
1/1 [==============================] - 0s 127ms/step
5
1/1 [==============================] - 0s 134ms/step
6
1/1 [==============================] - 0s 133ms/step
7
1/1 [==============================] - 0s 125ms/step
8
1/1 [==============================] - 0s 108ms/step


1/1 [==============================] - 0s 127ms/step
9
2/2 [==============================] - 0s 9ms/step
10
2/2 [==============================] - 0s 7ms/step
11
1/1 [==============================] - 0s 127ms/step
12
1/1 [==============================] - 0s 123ms/step
13
1/1 [==============================] - 0s 126ms/step
14
1/1 [==============================] - 0s 122ms/step
15
1/1 [==============================] - 0s 124ms/step
16
1/1 [==============================] - 0s 123ms/step
17
1/1 [==============================] - 0s 122ms/step
18
1/1 [==============================] - 0s 119ms/step
19
1/1 [==============================] - 0s 26ms/step


1/1 [==============================] - 0s 128ms/step
20
1/1 [==============================] - 0s 147ms/step
21
1/1 [==============================] - 0s 146ms/step
22
1/1 [==============================] - 0s 132ms/step
23
1/1 [==============================] - 0s 129ms/step
24
1/1 [==============================] - 0s 126ms/step
25
1/1 [==============================] - 0s 128ms/step
26
1/1 [==============================] - 0s 123ms/step
27
1/1 [==============================] - 0s 130ms/step
28
1/1 [==============================] - 0s 123ms/step
29
1/1 [==============================] - 0s 22ms/step


1/1 [==============================] - 0s 129ms/step
30
1/1 [==============================] - 0s 121ms/step
31
1/1 [==============================] - 0s 125ms/step
32
1/1 [==============================] - 0s 122ms/step
33
1/1 [==============================] - 0s 124ms/step
34
1/1 [==============================] - 0s 149ms/step
35
1/1 [==============================] - 0s 131ms/step
36
1/1 [==============================] - 0s 20ms/step


1/1 [==============================] - 0s 126ms/step
37
1/1 [==============================] - 0s 124ms/step
38
2/2 [==============================] - 0s 10ms/step
39
1/1 [==============================] - 0s 124ms/step
40
1/1 [==============================] - 0s 122ms/step
41
1/1 [==============================] - 0s 122ms/step
42
1/1 [==============================] - 0s 122ms/step
43
1/1 [==============================] - 0s 121ms/step
44
1/1 [==============================] - 0s 127ms/step
45
1/1 [==============================] - 0s 19ms/step


...

1/1 [==============================] - 0s 122ms/step
135
1/1 [==============================] - 0s 129ms/step
136
1/1 [==============================] - 0s 131ms/step
137
1/1 [==============================] - 0s 126ms/step
138
1/1 [==============================] - 0s 167ms/step
139
2/2 [==============================] - 0s 8ms/step
140
1/1 [==============================] - 0s 144ms/step
141
1/1 [==============================] - 0s 126ms/step
142
1/1 [==============================] - 0s 123ms/step
143
1/1 [==============================] - 0s 124ms/step
144
1/1 [==============================] - 0s 124ms/step


1/1 [==============================] - 0s 124ms/step
145
1/1 [==============================] - 0s 126ms/step
146
1/1 [==============================] - 0s 124ms/step
147
1/1 [==============================] - 0s 126ms/step
148
1/1 [==============================] - 0s 132ms/step
149
1/1 [==============================] - 0s 121ms/step
150
1/1 [==============================] - 0s 163ms/step
151
1/1 [==============================] - 0s 151ms/step
152
1/1 [==============================] - 0s 30ms/step


1/1 [==============================] - 0s 144ms/step
153
1/1 [==============================] - 0s 141ms/step
154
1/1 [==============================] - 0s 140ms/step
155
1/1 [==============================] - 0s 129ms/step
156
1/1 [==============================] - 0s 125ms/step
157
1/1 [==============================] - 0s 121ms/step
158
1/1 [==============================] - 0s 124ms/step
159
1/1 [==============================] - 0s 121ms/step
160
1/1 [==============================] - 0s 19ms/step


1/1 [==============================] - 0s 124ms/step
161
1/1 [==============================] - 0s 168ms/step
162
1/1 [==============================] - 0s 169ms/step
163
1/1 [==============================] - 0s 143ms/step
164
1/1 [==============================] - 0s 142ms/step
165
1/1 [==============================] - 0s 133ms/step
166
1/1 [==============================] - 0s 125ms/step
167
1/1 [==============================] - 0s 125ms/step
168
1/1 [==============================] - 0s 21ms/step


1/1 [==============================] - 0s 134ms/step
169
1/1 [==============================] - 0s 131ms/step
170
1/1 [==============================] - 0s 133ms/step
171
1/1 [==============================] - 0s 128ms/step
172
1/1 [==============================] - 0s 127ms/step
173
1/1 [==============================] - 0s 165ms/step
174
1/1 [==============================] - 0s 155ms/step
175
